In [1]:
!python --version

Python 3.8.10


In [ ]:
# !poetry remove pandas

In [11]:

# !poetry add tensorflow=2.5 tensorflow-text=2.5 tensorflow-hub 
# numpy=1.19.3 
# pandas==1.2.5 openpyxl

!poetry add pandas==1.2.5 openpyxl

The following packages are already present in the pyproject.toml and will be skipped:

  • openpyxl

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.


Updating dependencies
Resolving dependencies...

Writing lock file

Package operations: 1 install, 0 updates, 0 removals

  • Installing pandas (1.2.5)


In [2]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

In [3]:
# Some texts of different lengths.
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
italian_sentences = ["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."]
japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [4]:
# Compute embeddings.
en_result = embed(english_sentences)
it_result = embed(italian_sentences)
ja_result = embed(japanese_sentences)

In [1]:
en_result.numpy()[0]

NameError: name 'en_result' is not defined

In [6]:
# Compute similarity matrix. Higher score indicates greater similarity.
similarity_matrix_it = np.inner(en_result, it_result)
similarity_matrix_ja = np.inner(en_result, ja_result)

In [7]:
similarity_matrix_it

array([[0.9578779 , 0.3308629 , 0.30247942],
       [0.38761464, 0.73392963, 0.24814035],
       [0.2361753 , 0.21800497, 0.9283011 ]], dtype=float32)

In [12]:
import pandas as pd

In [13]:
df = pd.read_excel("ЭТ_Новости.xlsx")
df.head()

,ID,NEWS_HEAD
0,1552621575,Государственная Дума рекомендует Правительству...
1,1552115547,Евгений Грабчак: «Меры поддержки помогут ТЭК о...
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...
3,1551704812,Вниманию гарантирующих поставщиков и энергосбы...
4,1551704814,Эдуард Шереметцев: «Национальная энергетическа...


In [8]:
%%time
df["emb"] = df["NEWS_HEAD"].apply(lambda x: embed(x).numpy()[0])

Wall time: 10.8 s


In [9]:
df.to_pickle("../News+emb.pkl")